Датасет - https://www.kaggle.com/datasets/hijest/englishrussian-dictionary-for-machine-translate/data

#1. Установка необходимых библиотек

In [ ]:
!pip install evaluate
!pip install hf_xet
!pip install sacremoses
!pip install sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.

#2. Импорт библиотек

In [ ]:
import torch
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    pipeline,
    FSMTTokenizer,
    FSMTForConditionalGeneration
)
from evaluate import load
import pandas as pd
from tqdm import tqdm
import random
import numpy as np
from sacrebleu import BLEU

#3. Загрузка и подготовка данных

In [ ]:
text = "/content/rus.txt"
with open(text) as file:
    lines = file.read().split("\n")[:-1]

pairs = []
for line in lines:
    parts = line.strip().split("\t")
    if len(parts) >= 2:
        eng, rus = parts[0].strip(), parts[1].strip()
        pairs.append((eng, rus))

df = pd.DataFrame(pairs)
df.head(5)

print("Всего пар =", len(pairs))

# Использование только части данных
test_pairs = pairs
print("Используем только", len(test_pairs))

Всего пар = 363386
Используем только 363386


#4. Список моделей для оценки

In [ ]:
models = [
    "Helsinki-NLP/opus-mt-en-ru",
    "google-t5/t5-base",
    "facebook/wmt19-en-ru",
]

#5. Функция для оценки BLEU

In [ ]:
def evaluate_bleu_fast(model_name, test_pairs, sample_size=10000, batch_size=32, max_length=128):
    sacre_bleu = BLEU()

    # Выбор подвыборки данных
    if len(test_pairs) > sample_size:
        indices = np.random.choice(len(test_pairs), sample_size, replace=False)
        test_pairs = [test_pairs[i] for i in indices]

    try:
        # Определение типа модели и соответствующей обработки
        if "opus-mt" in model_name.lower():
            # Оптимизация для Helsinki-NLP
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model.to(device)

            pipe = pipeline(
                "translation",
                model=model,
                tokenizer=tokenizer,
                device=device
            )

            sources = [src for src, _ in test_pairs]
            predictions = []
            for out in tqdm(pipe(sources, batch_size=batch_size, max_length=max_length),
                          desc=f"Processing {model_name}"):
                predictions.append(out['translation_text'])

        elif "t5" in model_name.lower():
            # Обработка для T5 моделей
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = AutoModelForSeq2SeqLM.from_pretrained(
                model_name,
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
            )
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model.to(device)

            predictions = []
            with torch.no_grad(), torch.autocast("cuda" if torch.cuda.is_available() else "cpu"):
                for i in tqdm(range(0, len(test_pairs), batch_size), desc=f"Processing {model_name}"):
                    batch = test_pairs[i:i+batch_size]
                    sources, targets = zip(*batch)
                    inputs = [f"translate to ru: {src}" for src in sources]

                    tokenized = tokenizer(
                        inputs,
                        return_tensors="pt",
                        padding=True,
                        truncation=True,
                        max_length=max_length
                    ).to(model.device)

                    outputs = model.generate(
                        **tokenized,
                        max_new_tokens=max_length,
                        num_beams=1,
                        do_sample=False
                    )
                    predictions.extend(tokenizer.batch_decode(outputs, skip_special_tokens=True))

        elif "wmt19" in model_name.lower():
            # Обработка для моделей Facebook WMT19
            tokenizer = FSMTTokenizer.from_pretrained(model_name)
            model = FSMTForConditionalGeneration.from_pretrained(model_name)
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model.to(device)

            predictions = []
            with torch.no_grad():
                for i in tqdm(range(0, len(test_pairs), batch_size), desc=f"Processing {model_name}"):
                    batch = test_pairs[i:i+batch_size]
                    sources = [src for src, _ in batch]

                    input_ids = tokenizer(
                        sources,
                        return_tensors="pt",
                        padding=True,
                        truncation=True,
                        max_length=max_length
                    ).input_ids.to(device)

                    outputs = model.generate(input_ids, max_length=max_length)
                    predictions.extend(tokenizer.batch_decode(outputs, skip_special_tokens=True))

        else:
            raise ValueError(f"Unsupported model type: {model_name}")

        references = [tgt for _, tgt in test_pairs]
        return sacre_bleu.corpus_score(predictions, [references]).score

    except Exception as e:
        print(f"Error evaluating {model_name}: {str(e)}")
        return 0.0

#6. Оценка моделей и сравнение

In [ ]:
bleu_scores = {}

# Оценка BLEU для всех моделей
for model in models:
    bleu_score = evaluate_bleu_fast(model, test_pairs)
    bleu_scores[model] = bleu_score
    print(f"Модель {model} имеет BLEU: {bleu_score}")

# Сортировка моделей по BLEU
sorted_models = sorted(bleu_scores.items(), key=lambda x: x[1], reverse=True)

# Вывод топ-3 моделей
print("\nТоп-3 модели:")
for model, bleu in sorted_models[:3]:
    print(f"Модель: {model}, BLEU: {bleu}")

# Выбор лучшей модели
best_model = sorted_models[0][0]
print(f"\nЛучшая модель: {best_model}")

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

Processing facebook/wmt19-en-ru:   0%|          | 0/313 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.16G [00:00<?, ?B/s]

Processing facebook/wmt19-en-ru: 100%|██████████| 313/313 [02:35<00:00,  2.01it/s]


Модель facebook/wmt19-en-ru имеет BLEU: 42.20461008133821

Топ-3 модели:
Модель: Helsinki-NLP/opus-mt-en-ru, BLEU: 50.5747128824216
Модель: facebook/wmt19-en-ru, BLEU: 42.20461008133821
Модель: google-t5/t5-base, BLEU: 0.05434727996520763

Лучшая модель: Helsinki-NLP/opus-mt-en-ru


#7. Загрузка и использование лучшей модели

In [ ]:
# Загрузка лучшей модели
best_model_loaded = AutoModelForSeq2SeqLM.from_pretrained(best_model)
best_tokenizer = AutoTokenizer.from_pretrained(best_model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_model_loaded.to(device)

# Функция для перевода
def translate(text):
    input_ids = best_tokenizer.encode(text, return_tensors="pt").to(device)
    output = best_model_loaded.generate(input_ids)
    return best_tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
# Интерактивный перевод
print("\nИнтерактивный перевод (для выхода введите 'exit')")
while True:
    text = input("Введите текст на английском для перевода на русский: ")
    if text.lower() == 'exit':
        break
    translated_text = translate(text)
    print(f"Перевод: {translated_text}\n")


Интерактивный перевод (для выхода введите 'exit')
Введите текст на английском для перевода на русский: What's the weather like outside?
Перевод: Какая погода снаружи?

Введите текст на английском для перевода на русский: This is a model for text translation.
Перевод: Это модель для перевода текста.

Введите текст на английском для перевода на русский: How well does she translate text?
Перевод: Насколько хорошо она переводит смс?

Введите текст на английском для перевода на русский: exit
